# ETL da camada bronze para camada silver

Este notebook realiza o ETL dos dados da camada bronze para a camada silver. Ou seja: ele abre o dataset e o salva num dataframe, realiza a transformação dos dados e os carrega num arquivo `.csv` e no banco de dados.

## EXTRACT
# Estratégia de Leitura Escalável (PySpark)

Para contornar as limitações de memória (especialmente em ambientes como WSL) ao ler o arquivo `MICRODADOS_ENEM_2021.csv` (>1.5GB), adotamos a estratégia de processamento distribuído com **Apache Spark**.

Diferente do padrão do Pandas que tenta carregar todo o arquivo na RAM de uma vez, o PySpark utiliza o conceito de *Lazy Evaluation* (avaliação preguiçosa). Ele mapeia o arquivo e cria um plano de execução, mas só processa os dados na memória quando uma ação é solicitada, evitando o travamento do sistema.

#### **Parâmetros Críticos:**

* **`inferSchema="true"`:** Permite que o Spark percorra os dados inicialmente para identificar automaticamente quais colunas são numéricas e quais são textuais, facilitando a análise imediata.
* **`encoding="ISO-8859-1"`:** Corrige erros de decodificação de caracteres que contem dentro do arquivo `MICRODADOS_ENEM_2021.csv` que e um "erro" bem comuns em dados  usado no Brasil.
* **`delimiter=";"`:** Define o ponto e vírgula como o separador correto, evitando que o dataset seja interpretado equivocadamente como uma única coluna longa.

In [22]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("RawToSilves") \
    .config("spark.driver.memory", "5g") \
    .getOrCreate()

data_layer_filepath = '../raw/'

df = spark.read \
    .option("header", "true") \
    .option("delimiter", ";") \
    .option("encoding", "ISO-8859-1") \
    .option("inferSchema", "true") \
    .csv(data_layer_filepath + 'data_raw/MICRODADOS_ENEM_2021.csv')

print("Arquivo completo mapeado com sucesso!")

df.show(5)


Arquivo completo mapeado com sucesso!
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RAC

## TRANSFORM
### Padronização dos Nomes das Colunas.

Inicialmente, o dataset contém colunas com nomes sem padrão, por "sorte" nessa basse que estamos usando todos os nome estão padronizado porem tudo com letras Maisculas. E para fins de tratamento iremos fazer uma varedura para que caso ajá alguma coluna que não eteja separam palavras com `_`, ou tenha alguma coluna com `whitespace`, passse para o padrão com as colunas seguindo esse novo padrão: **todos os caracteres em minúsculo, separando palavras com `_`**.


Para renomear todas as colunas de uma vez convertendo para minúsculas e trocando espaços por `_`, a maneira mais eficiente quando se usa o **PySpark** é usar o método `.toDF().`

In [23]:

novas_colunas = [col.lower().replace(' ', '_') for col in df.columns]

df = df.toDF(*novas_colunas)

print(df.columns)
df.show(5)

['nu_inscricao', 'nu_ano', 'tp_faixa_etaria', 'tp_sexo', 'tp_estado_civil', 'tp_cor_raca', 'tp_nacionalidade', 'tp_st_conclusao', 'tp_ano_concluiu', 'tp_escola', 'tp_ensino', 'in_treineiro', 'co_municipio_esc', 'no_municipio_esc', 'co_uf_esc', 'sg_uf_esc', 'tp_dependencia_adm_esc', 'tp_localizacao_esc', 'tp_sit_func_esc', 'co_municipio_prova', 'no_municipio_prova', 'co_uf_prova', 'sg_uf_prova', 'tp_presenca_cn', 'tp_presenca_ch', 'tp_presenca_lc', 'tp_presenca_mt', 'co_prova_cn', 'co_prova_ch', 'co_prova_lc', 'co_prova_mt', 'nu_nota_cn', 'nu_nota_ch', 'nu_nota_lc', 'nu_nota_mt', 'tx_respostas_cn', 'tx_respostas_ch', 'tx_respostas_lc', 'tx_respostas_mt', 'tp_lingua', 'tx_gabarito_cn', 'tx_gabarito_ch', 'tx_gabarito_lc', 'tx_gabarito_mt', 'tp_status_redacao', 'nu_nota_comp1', 'nu_nota_comp2', 'nu_nota_comp3', 'nu_nota_comp4', 'nu_nota_comp5', 'nu_nota_redacao', 'q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009', 'q010', 'q011', 'q012', 'q013', 'q014', 'q015', 'q016', 

### Remoção de Colunas Desnecessárias

Afim de melhorar como os dados vão ficar nessa etapa começamos a filtrar e retirar alguns dados são desnecesarios como por exemplo **nu_ano** já que toda a base e referente ao ano de 2021, e assim optamos por não trabalhar com essa coluna.

In [24]:
cols_to_drop = ['nu_ano']

df = df.drop(*cols_to_drop) 

for col in cols_to_drop:
    if col not in df.columns:
        print(f"Coluna {col} deletada!")

print("Colunas restantes: ")
print(df.columns)
print(" ")
print("Tabelas restantes: ")
df.show(5)

Coluna nu_ano deletada!
Colunas restantes: 
['nu_inscricao', 'tp_faixa_etaria', 'tp_sexo', 'tp_estado_civil', 'tp_cor_raca', 'tp_nacionalidade', 'tp_st_conclusao', 'tp_ano_concluiu', 'tp_escola', 'tp_ensino', 'in_treineiro', 'co_municipio_esc', 'no_municipio_esc', 'co_uf_esc', 'sg_uf_esc', 'tp_dependencia_adm_esc', 'tp_localizacao_esc', 'tp_sit_func_esc', 'co_municipio_prova', 'no_municipio_prova', 'co_uf_prova', 'sg_uf_prova', 'tp_presenca_cn', 'tp_presenca_ch', 'tp_presenca_lc', 'tp_presenca_mt', 'co_prova_cn', 'co_prova_ch', 'co_prova_lc', 'co_prova_mt', 'nu_nota_cn', 'nu_nota_ch', 'nu_nota_lc', 'nu_nota_mt', 'tx_respostas_cn', 'tx_respostas_ch', 'tx_respostas_lc', 'tx_respostas_mt', 'tp_lingua', 'tx_gabarito_cn', 'tx_gabarito_ch', 'tx_gabarito_lc', 'tx_gabarito_mt', 'tp_status_redacao', 'nu_nota_comp1', 'nu_nota_comp2', 'nu_nota_comp3', 'nu_nota_comp4', 'nu_nota_comp5', 'nu_nota_redacao', 'q001', 'q002', 'q003', 'q004', 'q005', 'q006', 'q007', 'q008', 'q009', 'q010', 'q011', 'q012'

### Correções dos Tipos de Dados.

Nesta etapa, analisaremos a estrutura e a **tipagem dos dados** nas colunas para identificar inconsistências e realizar os tratamentos necessários.

In [25]:
# nome_arquivo = 'schema_enem.txt'
# f = open(nome_arquivo, 'w')

header = f"{'COLUNA':<40} | {'TIPO'}"
separador = "-" * 50
print(header)
print(separador)

# f.write(header + "\n")
# f.write(separador + "\n")

for coluna, tipo in df.dtypes:
    linha = f"{coluna:<40} | {tipo}"
    print(linha)
    
    # f.write(linha + "\n")

# f.close() 
# print(f"\nO arquivo '{nome_arquivo}' também foi salvo na pasta.")

COLUNA                                   | TIPO
--------------------------------------------------
nu_inscricao                             | bigint
tp_faixa_etaria                          | int
tp_sexo                                  | string
tp_estado_civil                          | int
tp_cor_raca                              | int
tp_nacionalidade                         | int
tp_st_conclusao                          | int
tp_ano_concluiu                          | int
tp_escola                                | int
tp_ensino                                | int
in_treineiro                             | int
co_municipio_esc                         | int
no_municipio_esc                         | string
co_uf_esc                                | int
sg_uf_esc                                | string
tp_dependencia_adm_esc                   | int
tp_localizacao_esc                       | int
tp_sit_func_esc                          | int
co_municipio_prova                       | 